In [ ]:
import urllib.request, zipfile
import pandas as pd, numpy as np,xarray as xr
from pathlib import Path
import re, yaml, copy, json
import helper, config_adapter
from helper import RenderJSON
import scipy.io.wavfile

In [ ]:
import itables
itables.init_notebook_mode(all_interactive=True )
itables.options.maxBytes = "1MB"
itables.options.lengthMenu = [25, 10, 50, 100, 200]
itables.options.buttons = ["copyHtml5", "csvHtml5", "excelHtml5"]
itables.options.layout={"topEnd": "pageLength", "top1": "searchBuilder"}

In [ ]:
params = yaml.safe_load(Path("params.yaml").open("r"))
RenderJSON(params)

In [ ]:
files = pd.DataFrame(params["files"])
files

In [ ]:
config_path = Path(params["config_path"])
config = config_adapter.load(config_path)
RenderJSON(config)

In [ ]:
import random
if "random_seed" in config:
    random.seed(config["random_seed"])

In [ ]:
annotations = []
for row in files.to_dict(orient="index").values():
    annotations.append(pd.read_csv(row["annotation_path"]).rename(columns={"name": "label", "start_seconds": "start", "stop_seconds": "end"}).assign(audio_path=row["audio_path"], file=row["annotation_path"]))
annotations = pd.concat(annotations).sort_values(["audio_path", "start"])
annotations

In [ ]:
block_shoulder_duration = config["processing"]["block_shoulder_duration"]
annotations["block_change"] = (annotations["file"] != annotations["file"].shift(1)) | ((annotations["start"] - 2*block_shoulder_duration) > annotations["end"].shift(1))
annotations["block"] = annotations["block_change"].cumsum()
annotations = annotations.drop(columns="block_change")
annotations


In [ ]:

groups = [df for _, df in annotations.groupby('block')]
random.shuffle(groups)
ars=[]
t_prev=0
common_fs = None
i=0
rows = []
for df in groups:
    fs, data = scipy.io.wavfile.read(df["audio_path"].iat[0], mmap=True)
    if common_fs is None: common_fs = fs
    elif fs!=common_fs: raise Exception("Not same fs")

    istart = int((max(df["start"].min()-block_shoulder_duration, 0))*fs)
    iend = min(int((df["end"].max()+block_shoulder_duration)*fs), data.size)

    ar = xr.Dataset()
    ar["data"] = xr.DataArray(data[istart: iend], dims="t")
    ar["t"] = np.arange(ar["data"].size)/fs + t_prev
    ar["file"] = df["audio_path"].iat[0]
    ar["t_file"] = xr.DataArray(np.arange(istart, iend)/fs, dims="t")
    ar["block"] = df["block"].iat[0]
    ar["label"] = xr.DataArray(np.full(ar["t"].size, "noise", dtype=str), dims="t")
    ar["syb_num"] = xr.DataArray(np.full(ar["t"].size, -1), dims="t")
    for _, row in df.to_dict(orient="index").items():
        ar["label"] = xr.where((ar["t_file"] >= row["start"]) & (ar["t_file"] <= row["end"]), row["label"], ar["label"])
        ar["syb_num"] = xr.where((ar["t_file"] >= row["start"]) & (ar["t_file"] <= row["end"]), i, ar["syb_num"])
        # row["syb_num"] = i
        row["new_t_start"] = row["start"] - (df["start"].min() - block_shoulder_duration) + t_prev
        row["new_t_end"] = row["end"] - (df["start"].min() - block_shoulder_duration) + t_prev
        i+=1
        rows.append(row)
        
    t_prev += ar["data"].size/fs
    ars.append(ar)
        
data: xr.Dataset = xr.concat(ars, dim="t")
data["t"].attrs["fs"] = common_fs
df = pd.DataFrame(rows)



In [ ]:
max_syb = len(df.index)
if np.abs(np.sum(list(config["processing"]["split"].values())) -1) > 10**(-5):
    raise Exception("problem sum!=1")
df["goal"] = "none"
df["syb_num"] = np.arange(len(df.index))
cur=0
for k,v in config["processing"]["split"].items():
    n = cur+ max_syb*v
    df["goal"] = np.where((df["syb_num"] >=cur) & (df["syb_num"] < n), str(k), df["goal"])
    cur=n
    


In [ ]:
stats = df.groupby(["goal", "label"]).apply(lambda d: pd.Series(dict(count=len(d.index), avg_duration=(d["end"] - d["start"]).mean())), include_groups=False).reset_index()
totals = df.groupby("label").apply(lambda d: pd.Series(dict(count=len(d.index), avg_duration=(d["end"] - d["start"]).mean())), include_groups=False).reset_index().assign(goal="all")
stats = pd.concat([stats, totals]).reset_index(drop=True).set_index(["goal", "label"]).unstack("goal")
stats["count"] = stats["count"].fillna(0)
stats

In [ ]:
out_annotations = df.rename(columns=dict(label="name", new_t_start="start_seconds", new_t_end="stop_seconds", start="source_start", end="source_end", audio_path="source_audio"))
out_annotations.to_csv(params["out_annotations"], index=False)
display(out_annotations)


In [ ]:
scipy.io.wavfile.write(params["out_song"], fs, data["data"].to_numpy())